In [18]:
#!pip install recordlinkage==0.14
import recordlinkage
import pandas as pd
import recordlinkage as rl 
from recordlinkage.index import Block
from recordlinkage.algorithms.distance import _haversine_distance
from recordlinkage.algorithms.numeric import _linear_sim
import random

In [3]:
#Reading Data
df_clean = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_red.csv')

#There was a duplicated entry in ListingId so delete
df_clean[df_clean.duplicated(subset=['ListingId']) == True]
df_clean.drop(132,axis=0,inplace=True)
df_clean[df_clean['ListingId']==10217568]


#To have tuples with LitingsId as index ,useful for later processing
df_tuple = df_clean.set_index('ListingId')

In [4]:
# To compare Harversine we need lat and ln in a tuple
df_clean['Lat_Lng'] =list(zip(df_clean.Lat, df_clean.Lng))

In [5]:
#Blocking of data , feature Neighborhood
indexer = rl.BlockIndex(on='Neighborhood')
candidate_links = indexer.index(df_tuple)

print ('Candidate links:',len(candidate_links))

Candidate links: 3470454


In [6]:
#Further blocking include longitude and latitude
compare_cl = rl.Compare()

compare_cl.exact('Neighborhood','Neighborhood', label='Neighborhood')
compare_cl.geo('Lat','Lng','Lat','Lng',method='linear',label='Distance')

features = compare_cl.compute(candidate_links, df_tuple)
features

Neighborhood  Distance
ListingId_1 ListingId_2                        
10025641    10008030                1  0.000000
10027284    10008030                1  0.331520
            10025641                1  0.000000
10043980    10008030                1  0.000000
            10025641                1  0.972364
...                               ...       ...
42112141    39267402                1  0.000000
49161472    30077792                1  0.000000
            39267402                1  0.000000
            42112141                1  0.000000
40629039    3292084                 1  0.000000

[3470454 rows x 2 columns]

In [7]:
#Select possible matches
matches_blocking = features[features['Distance'] > 0.925] # Threshold used accordig to bnb aprox distance
len(matches_blocking)

44337

In [19]:
#Stratified sampling according to literaure, top 100 most similar , bottom 100 
matches_blocking = matches_blocking.sort_values(by=['Distance'])
top100_matches_blocking = matches_blocking.iloc[-100:]
bottom100_matches_blocking = matches_blocking.iloc[:100]

#list with all the possible matching pairs 150k
top_pairs =list(top100_matches_blocking.index) 
bottom_pairs =list(bottom100_matches_blocking.index) 

In [23]:
#Sampling
sample_top = random.sample(top_pairs,3)
sample_bottom = random.sample(bottom_pairs,3)

In [29]:
labels = []

for id_1,id_2 in sample_top:
    url_1 =[x for x in df_clean['listing_url'][df_clean['ListingId'] == id_1]]
    url_2 =[x for x in df_clean['listing_url'][df_clean['ListingId'] == id_2]]
    print("Are this url's from the same listing?")
    print(url_1[0])
    print(url_2[0])
    label = int(input())
    labels.append(((id_1,id_2),label))
    print('Saved!')
for id_1,id_2 in sample_bottom:
    url_1 =[x for x in df_clean['listing_url'][df_clean['ListingId'] == id_1]]
    url_2 =[x for x in df_clean['listing_url'][df_clean['ListingId'] == id_2]]
    print("Are this url's from the same listing?")
    print(url_1[0])
    print(url_2[0])
    label = int(input())
    labels.append(((id_1,id_2),label))
    print('Saved!')


Are this url's from the same listing?
https://www.vrbo.com/nl-nl/vakantiewoning/p8772061?noDates=true
https://www.vrbo.com/nl-nl/vakantiewoning/p8772056?noDates=true
Saved!
Are this url's from the same listing?
https://www.airbnb.nl/rooms/13092978
https://www.airbnb.nl/rooms/12451293
Saved!
Are this url's from the same listing?
https://www.airbnb.nl/rooms/48127657
https://www.airbnb.nl/rooms/48127644
Saved!
Are this url's from the same listing?
https://www.vrbo.com/nl-nl/vakantiewoning/p364081vb?noDates=true
https://www.airbnb.nl/rooms/24296667
Saved!
Are this url's from the same listing?
https://www.airbnb.nl/rooms/9308904
https://www.airbnb.nl/rooms/53655681
Saved!
Are this url's from the same listing?
https://www.airbnb.nl/rooms/46561482
https://www.airbnb.nl/rooms/22055478
Saved!


In [53]:
labels

[((8772061, 8772056), 1),
 ((13092978, 12451293), 1),
 ((48127657, 48127644), 0),
 ((364081, 24296667), 0),
 ((9308904, 53655681), 0),
 ((46561482, 22055478), 0)]